In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import paper_utils as pu

pu.set_plots()

# Load data

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

## With and without weekends

In [ ]:
cooling_models_with_weekends = {}

weekend = True

f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
ax = ax.flatten()
for i, project in enumerate(pu.PROJECTS):
    pu.plot_base(data[project], fax=(f, ax[i]), weekend=weekend)
    cooling_models_with_weekends[project] = pu.create_model(
        data[project], fax=(f, ax[i]), weekend=weekend
    )
    ax[i].set_title(project)
for a in ax:
    a.xaxis.set_tick_params(labelbottom=True)
f.tight_layout()
pu.add_legend((f, ax))

cooling_models_no_weekends = {}

weekend = False

f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
ax = ax.flatten()
for i, project in enumerate(pu.PROJECTS):
    pu.plot_base(data[project], fax=(f, ax[i]), weekend=weekend)
    cooling_models_no_weekends[project] = pu.create_model(
        data[project], fax=(f, ax[i]), weekend=weekend
    )
    ax[i].set_title(project)
for a in ax:
    a.xaxis.set_tick_params(labelbottom=True)
f.tight_layout()
pu.add_legend((f, ax))

## Using the median actual setpoint instead of the scheduled setpoint

In [ ]:
cooling_models_med_sp = {}

f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
ax = ax.flatten()
for i, project in enumerate(pu.PROJECTS):
    weekend = pu.choose_weekend[project]
    pu.plot_base(data[project], fax=(f, ax[i]), weekend=weekend)
    cooling_models_med_sp[project] = pu.create_model(
        data[project], fax=(f, ax[i]), treated="med_sp_bin", weekend=weekend
    )
    ax[i].set_title(project)
for a in ax:
    a.xaxis.set_tick_params(labelbottom=True)
f.tight_layout()
pu.add_legend((f, ax))

## Additional weather variables

In [ ]:
cooling_models_TminTmax = {}
cooling_models_sun_rad = {}
cooling_models_wind = {}
cooling_models_RH = {}

for i, project in enumerate(pu.PROJECTS):
    weekend = pu.choose_weekend[project]
    cooling_models_TminTmax[project] = pu.create_model(
        data[project], x_variables=["Tmean", "Tmin", "Tmax"], weekend=weekend
    )
    cooling_models_sun_rad[project] = pu.create_model(
        data[project], x_variables=["Tmean", "sun_rad_mean"], weekend=weekend
    )
    cooling_models_wind[project] = pu.create_model(
        data[project], x_variables=["Tmean", "wind_mean"], weekend=weekend
    )
    cooling_models_RH[project] = pu.create_model(
        data[project], x_variables=["Tmean", "RH_mean"], weekend=weekend
    )

## Comparing the different models

In [ ]:
model_list = [
    cooling_models_no_weekends,
    cooling_models_with_weekends,
    cooling_models_med_sp,
    cooling_models_TminTmax,
    cooling_models_sun_rad,
    cooling_models_wind,
    cooling_models_RH,
]
names = [
    "(a) No weekends",
    "(b) With weekends",
    "(c) Actual med. setpoint",
    "(d) With Tmax/Tmin",
    "(e) With solar",
    "(f) With wind",
    "(g) With RH",
]

# f, ax = plt.subplots(1, 2, figsize=(PAGE_WIDTH*.47, ROW_HEIGHT*1.5), sharex=True)
f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 0.75))
ax = ax.flatten()

colors = [
    pu.custom_blue,
    pu.custom_orange,
    pu.custom_green,
    pu.cardinal_red,
    pu.sage,
    pu.dark_grey,
    pu.light_grey,
]

j = -0.3
for im, model in enumerate(model_list):
    for i, project in enumerate(pu.PROJECTS):
        pu.ci_plot(ax[0], i + j, model[project], "Treated", color=colors[im])
        pu.ci_plot(ax[1], i + j, model[project], "Tmean", color=colors[im])
    j += 0.1

for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=25, ha="right")
ax[0].set_ylabel("Impact (%)")
ax[1].set_ylabel("Impact (%/$\degree$F)")

ax[0].set_ylim(top=0)
ax[1].set_ylim(bottom=0)

title0 = (
    f"{pu.plot_numbers[0]}) "
    + "Impact of 2$\degree$F setpoint increase on energy for cooling"
)
title1 = f"{pu.plot_numbers[1]}) " + "Impact of mean daily OAT on energy for cooling"

ax[0].set_title(title0, ha="left", loc="left")
ax[1].set_title(title1, ha="left", loc="left")

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.87, bottom=0.4)

for i, name in enumerate(names):
    ax[-1].plot([], [], "o", color=colors[i], label=name, ms=2)
ax[-1].legend(loc=8, bbox_transform=f.transFigure, bbox_to_anchor=(0.5, 0.01), ncol=4)
if pu.save_fig:
    f.savefig(pu.si_path / "Figure S9-AB.pdf")

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 0.75))

j = -0.3
for im, model in enumerate(model_list):
    for i, project in enumerate(pu.PROJECTS):
        ax.bar(
            i + j,
            float(model[project].summary2().tables[0].loc[0, 3]),
            0.08,
            color=colors[im],
        )
    j += 0.1
ax.set_title(
    f"{pu.plot_numbers[2]}) " + "Model goodness-of-fit (Adjusted R$^2$)",
    ha="left",
    loc="left",
)
ax.set_xticks(range(len(pu.PROJECTS)))
ax.set_xticklabels(pu.PROJECTS, rotation=25, ha="right")
plt.subplots_adjust(top=0.85, bottom=0.25)
if pu.save_fig:
    f.savefig(pu.si_path / "Figure S9-C.pdf")